In [1]:
import os
os.getcwd()
os.chdir("../..")
os.getcwd()

'/home/caleb/Cornell-Conversational-Analysis-Toolkit'

In [2]:
import convokit
import pickle
import pandas as pd


In [3]:
from convokit import Corpus, Conversation, Utterance

In [4]:
fake_news_dir = '/sauna/fake-news'
politics_dir = '/sauna/reddit_201810_raw/corpus/pokemontrades_banlist~-~politics/politics'
donald_dir = '/sauna/reddit_201810_raw/corpus/TheTwoBeerQueers~-~The_Donald/The_Donald/'

In [5]:
fake_news_urls = {'breitbart.com', 'dcgazette.com', "dailymail.co.uk", 
                  "lewrockwell.com", "newswars.com", "gellerreport.com",
                  "dcclothesline.com", "thegatewaypundit.com", "trueactivist.com"
                 }

In [6]:
real_news_sites = ['BBC', 'Business Insider', 'Buzzfeed', 'CBS News', 'CNN', 'Daily Beast', 
                   'FT Westminster Blog', 'FiveThirtyEight', 'Fortune', 'Mercury News', 'NPR',
                   'National Review', 'New Yorker', 'Newsweek', 'PBS', 'Politico', 'Real Clear Politics', 'Reuters',
                   'Slate', 'The American Conservative', 'The Denver Post', 'The Guardian', 'The Hill', 'The New York Times',
                   'The Verge', 'USA Today', 'Vox', 'WSJ Washington Wire', 'Washington Monthly', 'Washington Post']

In [7]:
real_news_urls = {'bbc.com', 'businessinsider.com', 'buzzfeed.com', 'cbsnews.com', 'cnn.com', 'thedailybeast.com',
                  'ft.com', 'fivethirtyeight.com', 'fortune.com', 'mercurynews.com', 'npr.org', 'nationalreview.com',
                  'newyorker.com', 'newsweek.com', 'pbs.org', 'politico.com', 'realclearpol', 'reuters.com',
                  'slate.com', 'theamericanconservative.com', 'denverpost.com', 'theguardian.com', 'thehill.com',
                  'nytimes.com', 'theverge.com', 'usatoday.com', 'vox.com', 'wsj.com', 'washingtonmonthly.com', 
                  'washingtonpost.com'
                 }

In [8]:
# politics_corpus = Corpus(filename=os.path.join(fake_news_dir, 'politics_corpus_paired_2year'))
politics_corpus = Corpus(filename=politics_dir)
# donald_corpus = Corpus(filename=os.path.join(fake_news_dir, 'donald_corpus_valid_convos'))

In [9]:
# Start time: 01 Oct 2016
# End time: 01 Oct 2018
start_time = 1475280000
end_time = 1538352000

In [10]:
from collections import defaultdict
real_news_convos = defaultdict(lambda: defaultdict(list))
fake_news_convos = defaultdict(lambda: defaultdict(list))
for convo in politics_corpus.iter_conversations():
    if start_time <= convo.meta['timestamp'] <= end_time:
        if convo.meta['domain'] in real_news_urls:
            real_news_convos[convo.meta['domain']][convo.meta['url']].append(convo.id)
        elif convo.meta['domain'] in fake_news_urls:
            fake_news_convos[convo.meta['domain']][convo.meta['url']].append(convo.id)

In [11]:
thread_pfxs = politics_corpus.utterance_threads(prefix_len=5, include_root=False)
convos_with_valid_len = set()
convo_to_thread = defaultdict(list)
for thread_id, utts in thread_pfxs.items():
    if len(utts) >= 5:
        convos_with_valid_len.add(utts[next(iter(utts))].root)
        convo_to_thread[utts[next(iter(utts))].root].append(thread_id)

In [12]:
from random import choice

for convo, threads in convo_to_thread.items():
    convo_to_thread[convo] = choice(threads)

In [13]:
for domain, urls in real_news_convos.items():
    for url, convos in urls.items():
        valid_convos = [c for c in convos if c in convo_to_thread]
        urls[url] = choice(valid_convos) if len(valid_convos) > 0 else None

for domain, urls in fake_news_convos.items():
    for url, convos in urls.items():
        valid_convos = [c for c in convos if c in convo_to_thread]
        urls[url] = choice(valid_convos) if len(valid_convos) > 0 else None

In [14]:
for domain, urls in real_news_convos.items():
    real_news_convos[domain] = {url: convo_id for url, convo_id in urls.items() if convo_id is not None}

for domain, urls in fake_news_convos.items():
    fake_news_convos[domain] = {url: convo_id for url, convo_id in urls.items() if convo_id is not None}



In [15]:
real_news_convo_ids = []
fake_news_convo_ids = []
for domain, urls in real_news_convos.items():
    real_news_convo_ids.extend(list(urls.values()))
for domain, urls in fake_news_convos.items():
    fake_news_convo_ids.extend(list(urls.values()))

In [16]:
real_news_thread_ids = [convo_to_thread[convo_id] for convo_id in real_news_convo_ids]
fake_news_thread_ids = [convo_to_thread[convo_id] for convo_id in fake_news_convo_ids]



In [17]:
for k, v in real_news_convos.items():
    print("{}: {}".format(k, len(v)))
    
print("Total posts:", sum([len(v) for v in real_news_convos.values()]))

politico.com: 7980
thehill.com: 24961
washingtonpost.com: 20294
cnn.com: 9864
nytimes.com: 7846
buzzfeed.com: 1848
businessinsider.com: 4932
npr.org: 2275
reuters.com: 3970
usatoday.com: 3184
ft.com: 70
theguardian.com: 4692
fivethirtyeight.com: 553
nationalreview.com: 1171
thedailybeast.com: 4358
pbs.org: 551
vox.com: 4107
newyorker.com: 635
bbc.com: 1513
cbsnews.com: 2495
fortune.com: 621
slate.com: 2657
mercurynews.com: 221
wsj.com: 1023
theverge.com: 358
newsweek.com: 4451
theamericanconservative.com: 94
denverpost.com: 276
washingtonmonthly.com: 268
Total posts: 117268


In [18]:
for k, v in fake_news_convos.items():
    print("{}: {}".format(k, len(v)))
    
print("Total posts:", sum([len(v) for v in fake_news_convos.values()]))

breitbart.com: 3226
dailymail.co.uk: 877
thegatewaypundit.com: 4
lewrockwell.com: 10
trueactivist.com: 5
dcclothesline.com: 4
gellerreport.com: 0
newswars.com: 0
Total posts: 4126


In [19]:
len(real_news_thread_ids)

117268

In [20]:
len(fake_news_thread_ids)

4126

In [21]:
real_fake_valid_ids = set(real_news_convo_ids).union(set(fake_news_convo_ids))

In [22]:
politics_corpus.filter_conversations_by(func=lambda convo: convo.id in real_fake_valid_ids)

In [23]:
politics_corpus.print_summary_stats()

Number of Users: 581859
Number of Utterances: 18375782
Number of Conversations: 121394


In [24]:
politics_corpus.dump('politics_corpus_fake_real_valid_convos_1year', base_path=fake_news_dir)

In [25]:
from random import sample
real_news_thread_sample = sample(real_news_thread_ids, len(fake_news_thread_ids))

In [26]:
thread_to_convo = {thread_id: convo_id for convo_id, thread_id in convo_to_thread.items()}

In [27]:
real_news_convos_sample = [thread_to_convo[thread_id] for thread_id in real_news_thread_sample]

In [28]:
fake_news_long_convos = [thread_to_convo[thread_id] for thread_id in fake_news_thread_ids]

In [29]:
paired_convos = set(real_news_convos_sample).union(fake_news_long_convos)

In [30]:
convo_to_thread_filtered = {k: v for k, v in convo_to_thread.items() if k in paired_convos}

In [31]:
len(fake_news_thread_ids)

4126

In [32]:
balanced_threads = fake_news_thread_ids + real_news_thread_sample

In [33]:
len(set(balanced_threads))

8252

In [34]:
len(paired_convos)

8252

In [35]:
politics_corpus.filter_conversations_by(func=lambda convo: convo.id in paired_convos)

In [36]:
politics_corpus.dump('politics_corpus_paired_1year', base_path=fake_news_dir)

In [37]:
# donald_corpus = Corpus(filename=os.path.join(fake_news_dir, 'donald_corpus_paired'))

In [38]:
politics_corpus.print_summary_stats()

Number of Users: 119610
Number of Utterances: 802204
Number of Conversations: 8252


In [39]:
hc = convokit.HyperConvo(min_thread_len=5, prefix_len=5, include_root=False)

In [40]:
motif_feats = hc.retrieve_feats(politics_corpus)

In [41]:
balanced_threads = set(balanced_threads)

In [42]:
motif_feats = {k: v for k, v in motif_feats.items() if k in balanced_threads}

In [43]:
motif_counts_df = pd.DataFrame.from_dict(motif_feats, orient='index')
motif_feat_names = list(motif_counts_df.columns)

In [44]:
hyperconv_feats = hc.retrieve_feats(politics_corpus)
hyperconv_feats = {k: v for k, v in hyperconv_feats.items() if k in balanced_threads}

In [45]:
hyperconv_df = pd.DataFrame.from_dict(hyperconv_feats, orient='index')
hyperconv_feat_names = list(hyperconv_df.columns)

In [46]:
dyadic_feats = {k: v for k, v in hc.retrieve_dyadic_motif_counts(politics_corpus).items() if k in balanced_threads}

In [47]:
dyadic_df = pd.DataFrame.from_dict(dyadic_feats, orient='index')
dyadic_feat_names = list(dyadic_df.columns)

In [48]:
path_feats = hc.retrieve_motif_pathway_stats(politics_corpus)
path_feats = {k: v for k, v in path_feats.items() if k in balanced_threads}

In [49]:
path_stats = hc.retrieve_motif_pathway_stats(politics_corpus)
path_stats = {k: v for k, v in path_stats.items() if k in balanced_threads}

path_stats_df = pd.DataFrame.from_dict(path_stats, orient='index')
columns = ['PATH-'+', '.join(filter(lambda x: type(x) == str, col)).strip() for col in path_stats_df.columns.values]
path_stats_df.columns = columns

path_stats_enum_df = pd.DataFrame()

for path_stat in columns:
    path_stats_enum_df['is-present[{}]'.format(path_stat)] = path_stats_df[path_stat] > 0
    path_stats_enum_df['count[{}]'.format(path_stat)] = path_stats_df[path_stat]

path_feat_names = list(path_stats_enum_df.columns)

In [50]:
feats_df = pd.concat([hyperconv_df, path_stats_enum_df, dyadic_df, motif_counts_df], axis=1)

/home/caleb/miniconda/envs/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [51]:
feats_df['volume'] = [len(set(utt.user.name for utt in thread_pfxs[thread_id].values())) for thread_id in feats_df.index]

In [52]:
fake_news_thread_ids = set(fake_news_thread_ids)
y = [int(thread_id in fake_news_thread_ids) for thread_id in list(feats_df.index)]

In [53]:
import numpy as np
y = np.array(y)

In [54]:
feats_df['y'] = y

In [55]:
feats_df.shape

(8252, 269)

In [56]:
feats_df = feats_df.fillna(-1)

In [57]:
feats_df = feats_df.astype('float64')

In [58]:
def print_extreme_coefs(clf, feature_names, num_features: int = 5):
    coefs = clf.named_steps['logreg'].coef_[0].tolist()

    assert len(feature_names) == len(coefs)

    feats_coefs = sorted(list(zip(feature_names, coefs)), key=lambda x: x[1], reverse=True)

    print()
    print("TOP {} FEATURES".format(num_features))
    for ft, coef in feats_coefs[:num_features]:
        print("{}: {:.3f}".format(ft, coef))
    print()
    print("BOTTOM {} FEATURES".format(num_features))
    for ft, coef in feats_coefs[-num_features:]:
        print("{}: {:.3f}".format(ft, coef))
    print()

In [59]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut 
from sklearn.model_selection import cross_val_score


for feature_set, name in [(hyperconv_feat_names, "hyperconvo"),
                        (["volume"], "volume"),
                        (motif_feat_names, "motifs"),
                        (dyadic_feat_names, "dyadic"),
                        (path_feat_names, "motifpaths"),
                        (motif_feat_names + path_feat_names, "motif-all"),
                        (hyperconv_feat_names + motif_feat_names, "hyperconv-motif"),
                        (hyperconv_feat_names + path_feat_names, "hyperconv-paths"),
                        (hyperconv_feat_names + motif_feat_names + path_feat_names, "hyperconvo-motifall"),
                        (hyperconv_feat_names + motif_feat_names + path_feat_names + ["volume"], "hyperconvo-motifall-volume")
                       ]:
    clf = Pipeline([("standardScaler", StandardScaler()), ("logreg", LogisticRegression(solver='liblinear'))])      
    loo = LeaveOneOut()
    X = np.array(feats_df[feature_set])
    y = np.array(feats_df['y'])
    scores = cross_val_score(clf, X, y, cv=20)
    print("- {}, cv_accuracy: {:.4f}".format(name, scores.mean()))
#     print("- {}, train: {:.4f}, test: {:.4f}".format(name, train_score, test_score))
#     print_extreme_coefs(clf, feature_set, num_features = 5)
    

- hyperconvo, cv_accuracy: 0.5479
- volume, cv_accuracy: 0.5196
- motifs, cv_accuracy: 0.5479
- dyadic, cv_accuracy: 0.5332
- motifpaths, cv_accuracy: 0.5566
- motif-all, cv_accuracy: 0.5556
- hyperconv-motif, cv_accuracy: 0.5479
- hyperconv-paths, cv_accuracy: 0.5556
- hyperconvo-motifall, cv_accuracy: 0.5552
- hyperconvo-motifall-volume, cv_accuracy: 0.5556


In [ ]:
thread_pfxs = politics_corpus.utterance_threads(prefix_len=8, include_root=False)

## BoW prediction

In [ ]:
thread_pfx_texts = {k: " ".join([utt.text for utt in v.values()]) for k, v in thread_pfxs.items() if k in balanced_threads}

In [ ]:
y = [int(thread_id in real_news_thread_sample) for thread_id in list(feats_df.index)]

In [ ]:
train_ids, test_ids, y_train, y_test = train_test_split(list(balanced_threads), y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df=0.05, max_df=0.7, ngram_range=(1,3))

In [ ]:
X_train = cv.fit_transform([thread_pfx_texts[train_id] for train_id in train_ids]).toarray()

In [ ]:
X_test = cv.transform([thread_pfx_texts[test_id] for test_id in test_ids]).toarray()

In [ ]:
clf = Pipeline([("standardScaler", StandardScaler()), ("logreg", LogisticRegression(solver='liblinear'))])      

clf.fit(X_train, y_train)
train_score = clf.score(X_train, y_train)
test_score = clf.score(X_test, y_test)
print("- {}, train: {:.4f}, test: {:.4f}".format("BoW", train_score, test_score))


In [ ]:
print_extreme_coefs(clf, cv.get_feature_names(), num_features = 10)